In [1]:

from pathlib import Path
data_root = Path('../kkdata3')
for x in data_root.glob('*'):
    print(x)

..\kkdata3\label_test_source.parquet
..\kkdata3\label_train_source.parquet
..\kkdata3\label_train_target.parquet
..\kkdata3\meta_song.parquet
..\kkdata3\meta_song_composer.parquet
..\kkdata3\meta_song_genre.parquet
..\kkdata3\meta_song_lyricist.parquet
..\kkdata3\meta_song_producer.parquet
..\kkdata3\meta_song_titletext.parquet
..\kkdata3\sample.csv


In [2]:
import pandas as pd
train_source = pd.read_parquet(data_root / 'label_train_source.parquet')
train_target = pd.read_parquet(data_root / 'label_train_target.parquet')
test_source = pd.read_parquet(data_root / 'label_test_source.parquet')
meta_song = pd.read_parquet(data_root / 'meta_song.parquet')

In [3]:
test_source.sort_values(['session_id', 'listening_order'], inplace=True)

In [4]:
train_source.sort_values(['session_id', 'listening_order'], inplace=True)

In [5]:
for i in range(1, 3):
    test_source[f'next{i}_song_id'] = test_source['song_id'].shift(-i)
    train_source[f'next{i}_song_id'] = train_source['song_id'].shift(-i)

In [6]:
test_source['next2_session_id'] = test_source['session_id'].shift(-2)
train_source['next2_session_id'] = train_source['session_id'].shift(-2)

In [7]:
df0 = train_source.query('session_id == next2_session_id')[['song_id', 'next1_song_id', 'next2_song_id']]
df1 = test_source.query('session_id == next2_session_id')[['song_id', 'next1_song_id', 'next2_song_id']]
df = pd.concat([df0, df1], axis=0, ignore_index=True)

In [8]:
df

,song_id,next1_song_id,next2_song_id
0,f6f06a71bb8bc38af6c0b7dae9cab00d,7b48a87effd31c9c07b68ed212062854,61c46d6401aab1dde7c7de23dc55c037
1,7b48a87effd31c9c07b68ed212062854,61c46d6401aab1dde7c7de23dc55c037,7e54c9199aad70e35fe256d23701bad0
2,61c46d6401aab1dde7c7de23dc55c037,7e54c9199aad70e35fe256d23701bad0,6178580fa01b62e9b52787902c0d8ae6
3,7e54c9199aad70e35fe256d23701bad0,6178580fa01b62e9b52787902c0d8ae6,ab694649c65477d0bc574bf391a3f4a0
4,6178580fa01b62e9b52787902c0d8ae6,ab694649c65477d0bc574bf391a3f4a0,5b3387fa195672dcfe979d17e4a62c9e
...,...,...,...
12875809,e667f16939964f81bccad710ed0adce9,1353435a907399cb65b925c6b5e3960b,00cef2617cceaa2299a47a79f6100ee2
12875810,1353435a907399cb65b925c6b5e3960b,00cef2617cceaa2299a47a79f6100ee2,605b1abcba2f893bdceaf20276be07c7
12875811,00cef2617cceaa2299a47a79f6100ee2,605b1abcba2f893bdceaf20276be07c7,3df18462598942a0e906c9327fc0e738
12875812,605b1abcba2f893bdceaf20276be07c7,3df18462598942a0e906c9327fc0e738,3b4f31812ec47aaf14ab56939dbe9b57


In [9]:
df_freq = df.groupby(['song_id', 'next1_song_id']).value_counts(sort=True, normalize=True).reset_index(name='freq')
df_freq

,song_id,next1_song_id,next2_song_id,freq
0,00002d2ddb6c789bcb8ef14f52339e39,02ad92330fc483bc5aa7dba81aea2dbe,a200e3c003617a157284d7f59a52e386,0.666667
1,00002d2ddb6c789bcb8ef14f52339e39,02ad92330fc483bc5aa7dba81aea2dbe,ee2fb6084403c67249eb331783e730fc,0.333333
2,00002d2ddb6c789bcb8ef14f52339e39,7ae025da34fbb414d9beaa5bb5fef66e,1b8b6d30519419eb7b272fac7cffdfd4,1.000000
3,00002d2ddb6c789bcb8ef14f52339e39,bc74c4a61822fd7bc58e55a8b4159ae1,689333702ad97af27e22b40ba11614e5,1.000000
4,00005d588ff5b96c84556f17b716e227,4cfbb7a6084c21c3565b4b69065c3fdb,b3a64d0705689d65f402676999b6d3ed,1.000000
...,...,...,...,...
10095237,ffffcf62a064a729b55618a2648c6af0,29161155b8d60571e4948243ca29b98d,4eec9ff2cec05b9e978a9c202fc103e1,1.000000
10095238,ffffcf62a064a729b55618a2648c6af0,2da05fbed3e68d10ccf6c0939d7e52de,ece2c29d573c996c07199ade20a8d09b,1.000000
10095239,ffffcf62a064a729b55618a2648c6af0,d9779ba8a5733c542b644413164a3c0e,ece2c29d573c996c07199ade20a8d09b,1.000000
10095240,ffffe1b0071c87b3716f708f7066a22f,949ad11fe2f6e6fbf6228bbffb9b36fb,599f2b612dfe484625d686b69e772272,1.000000


In [10]:
df_predict = df_freq.sort_values(['song_id', 'next1_song_id', 'freq'], ascending=False).groupby(['song_id', 'next1_song_id']).head(1)
df_predict

,song_id,next1_song_id,next2_song_id,freq
10095241,ffffec8b691fb58debb2587b0e48c71f,b5a3cf61efaf108370719d1c2e5edd25,94ac23554609d19d2a5d9b1163e75f98,1.000000
10095240,ffffe1b0071c87b3716f708f7066a22f,949ad11fe2f6e6fbf6228bbffb9b36fb,599f2b612dfe484625d686b69e772272,1.000000
10095239,ffffcf62a064a729b55618a2648c6af0,d9779ba8a5733c542b644413164a3c0e,ece2c29d573c996c07199ade20a8d09b,1.000000
10095238,ffffcf62a064a729b55618a2648c6af0,2da05fbed3e68d10ccf6c0939d7e52de,ece2c29d573c996c07199ade20a8d09b,1.000000
10095237,ffffcf62a064a729b55618a2648c6af0,29161155b8d60571e4948243ca29b98d,4eec9ff2cec05b9e978a9c202fc103e1,1.000000
...,...,...,...,...
5,00008cfd4308f6abae6136217bc95e32,0df8c3fe1bbac6f81fa8c98580a618d6,00008cfd4308f6abae6136217bc95e32,1.000000
4,00005d588ff5b96c84556f17b716e227,4cfbb7a6084c21c3565b4b69065c3fdb,b3a64d0705689d65f402676999b6d3ed,1.000000
3,00002d2ddb6c789bcb8ef14f52339e39,bc74c4a61822fd7bc58e55a8b4159ae1,689333702ad97af27e22b40ba11614e5,1.000000
2,00002d2ddb6c789bcb8ef14f52339e39,7ae025da34fbb414d9beaa5bb5fef66e,1b8b6d30519419eb7b272fac7cffdfd4,1.000000


In [11]:
x0 = train_source.query('listening_order == 19')[['session_id', 'song_id']].set_index('session_id')
x1 = train_source.query('listening_order == 20')[['session_id', 'song_id']].set_index('session_id')
x1 = x0.join(x1, lsuffix='_0', rsuffix='_1')

In [12]:
x1

,song_id_0,song_id_1
session_id,,
1,7bb8fadfc8f2bf145f4b29a0325fe79a,824c159701c8553b0e38f0d36ddd6197
2,3030938d53f52426ac30f213cf9915a1,6309716e08a58c64871c823f44749686
3,e904a955c0350ca5b1b6bb84174ee5be,3b050c1502af19554bb9ef8efc67c00d
4,6650e87fecc3e969c0d48f7f5e9b2c81,e29a8f57507d7e4a926a6bd3d51841d3
5,c10efc962e6b703053c2b44b7d48e8da,fd96fd2df80abfae763c3693932f8b26
...,...,...
715317,0f59db4259ec62fcf00399b69786a3c0,cb6e848eff5f656c47a4541e6be72079
715320,477ecc7fb10644c0d31408030cd464c6,aa7ce52650c7e4f7979e087cedb89020
715321,9e65aa27a1c3060b5909dda36859f39f,fba1bc1cf12692b1dd65c98e79985e52


In [13]:
x2 = train_target.pivot_table(index='session_id', columns='listening_order', values='song_id', aggfunc='first')
x2

listening_order,21,22,23,24,25
session_id,,,,,
1,b186d853cb06ceba2bd56bcdc701b8aa,1071e128ee5f9e2a0f0be0aade025b39,0100af68f3477f6bf724664d0b303f29,22a696cede3609c20ba1bceb8de28032,d6476892c926c88c7f232fd800b75845
2,cd6dade76334f76d993b913d10bb2ac7,822486e25be33911a62b3f12e549d24d,ed37d73c29e3696f65817916316bf05d,197a95a09a55390eb435e35f7dab9f9f,59844ef9a5e469cd13b1208c4f1142d1
3,a17aab4730e49f8e2972bf26227b890c,868bbeb92bd33a728dcf77d0073cd44d,c4d98c14bd48ec0495239c1a65a5178c,4ea79c3619515a39423851b5ceca965e,d76c868bfb392d89b3ef222907de69fc
4,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167
5,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,b3ac505ffec601de591ce2ce2e010d30
...,...,...,...,...,...
715317,3c66c29e5c26efc705c261732b977950,a0c8441cbe902320c846da76329a38e6,5d043bbef0ca3cf5385432478f3d3751,3b48c6883b4d16ddc07cb64b7be38d1c,546da46c864f38bb193940a7b8704f44
715320,b4b1c6c0e7395c50f2cf0702dd1f85d6,dd45bfa90e9186681877c15eb0166a35,c366d245501f007629813a28c9267d11,f4aabaca763955330cdf57f763d2a72c,f4aabaca763955330cdf57f763d2a72c
715321,261bceb1e3440067b85f5c54444d91be,65961a80125282df5c95b4c0a3606cc5,a91ec5b2b51524c53ce866f621208087,ab8683d279c095064693c282b8353b98,01c48d6a895a6efed0d5afba83b87902


In [14]:
x2.columns

Index([21, 22, 23, 24, 25], dtype='int64', name='listening_order')

In [15]:
x2.rename(columns={i: f'song_id_{i}' for i in range(21, 26)}, inplace=True)
x2

listening_order,song_id_21,song_id_22,song_id_23,song_id_24,song_id_25
session_id,,,,,
1,b186d853cb06ceba2bd56bcdc701b8aa,1071e128ee5f9e2a0f0be0aade025b39,0100af68f3477f6bf724664d0b303f29,22a696cede3609c20ba1bceb8de28032,d6476892c926c88c7f232fd800b75845
2,cd6dade76334f76d993b913d10bb2ac7,822486e25be33911a62b3f12e549d24d,ed37d73c29e3696f65817916316bf05d,197a95a09a55390eb435e35f7dab9f9f,59844ef9a5e469cd13b1208c4f1142d1
3,a17aab4730e49f8e2972bf26227b890c,868bbeb92bd33a728dcf77d0073cd44d,c4d98c14bd48ec0495239c1a65a5178c,4ea79c3619515a39423851b5ceca965e,d76c868bfb392d89b3ef222907de69fc
4,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167
5,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,b3ac505ffec601de591ce2ce2e010d30
...,...,...,...,...,...
715317,3c66c29e5c26efc705c261732b977950,a0c8441cbe902320c846da76329a38e6,5d043bbef0ca3cf5385432478f3d3751,3b48c6883b4d16ddc07cb64b7be38d1c,546da46c864f38bb193940a7b8704f44
715320,b4b1c6c0e7395c50f2cf0702dd1f85d6,dd45bfa90e9186681877c15eb0166a35,c366d245501f007629813a28c9267d11,f4aabaca763955330cdf57f763d2a72c,f4aabaca763955330cdf57f763d2a72c
715321,261bceb1e3440067b85f5c54444d91be,65961a80125282df5c95b4c0a3606cc5,a91ec5b2b51524c53ce866f621208087,ab8683d279c095064693c282b8353b98,01c48d6a895a6efed0d5afba83b87902


In [16]:
x3 = x1.join(x2, how='inner', on='session_id')
x3

,song_id_0,song_id_1,song_id_21,song_id_22,song_id_23,song_id_24,song_id_25
session_id,,,,,,,
1,7bb8fadfc8f2bf145f4b29a0325fe79a,824c159701c8553b0e38f0d36ddd6197,b186d853cb06ceba2bd56bcdc701b8aa,1071e128ee5f9e2a0f0be0aade025b39,0100af68f3477f6bf724664d0b303f29,22a696cede3609c20ba1bceb8de28032,d6476892c926c88c7f232fd800b75845
2,3030938d53f52426ac30f213cf9915a1,6309716e08a58c64871c823f44749686,cd6dade76334f76d993b913d10bb2ac7,822486e25be33911a62b3f12e549d24d,ed37d73c29e3696f65817916316bf05d,197a95a09a55390eb435e35f7dab9f9f,59844ef9a5e469cd13b1208c4f1142d1
3,e904a955c0350ca5b1b6bb84174ee5be,3b050c1502af19554bb9ef8efc67c00d,a17aab4730e49f8e2972bf26227b890c,868bbeb92bd33a728dcf77d0073cd44d,c4d98c14bd48ec0495239c1a65a5178c,4ea79c3619515a39423851b5ceca965e,d76c868bfb392d89b3ef222907de69fc
4,6650e87fecc3e969c0d48f7f5e9b2c81,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167
5,c10efc962e6b703053c2b44b7d48e8da,fd96fd2df80abfae763c3693932f8b26,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,b3ac505ffec601de591ce2ce2e010d30
...,...,...,...,...,...,...,...
715317,0f59db4259ec62fcf00399b69786a3c0,cb6e848eff5f656c47a4541e6be72079,3c66c29e5c26efc705c261732b977950,a0c8441cbe902320c846da76329a38e6,5d043bbef0ca3cf5385432478f3d3751,3b48c6883b4d16ddc07cb64b7be38d1c,546da46c864f38bb193940a7b8704f44
715320,477ecc7fb10644c0d31408030cd464c6,aa7ce52650c7e4f7979e087cedb89020,b4b1c6c0e7395c50f2cf0702dd1f85d6,dd45bfa90e9186681877c15eb0166a35,c366d245501f007629813a28c9267d11,f4aabaca763955330cdf57f763d2a72c,f4aabaca763955330cdf57f763d2a72c
715321,9e65aa27a1c3060b5909dda36859f39f,fba1bc1cf12692b1dd65c98e79985e52,261bceb1e3440067b85f5c54444d91be,65961a80125282df5c95b4c0a3606cc5,a91ec5b2b51524c53ce866f621208087,ab8683d279c095064693c282b8353b98,01c48d6a895a6efed0d5afba83b87902


In [39]:
x4 = pd.merge(x3, df_predict, how='left', left_on=['song_id_0', 'song_id_1'], right_on=['song_id', 'next1_song_id'])
del x4['song_id']
del x4['next1_song_id']
x4

,song_id_0,song_id_1,song_id_21,song_id_22,song_id_23,song_id_24,song_id_25,next2_song_id,freq
0,7bb8fadfc8f2bf145f4b29a0325fe79a,824c159701c8553b0e38f0d36ddd6197,b186d853cb06ceba2bd56bcdc701b8aa,1071e128ee5f9e2a0f0be0aade025b39,0100af68f3477f6bf724664d0b303f29,22a696cede3609c20ba1bceb8de28032,d6476892c926c88c7f232fd800b75845,NaN,NaN
1,3030938d53f52426ac30f213cf9915a1,6309716e08a58c64871c823f44749686,cd6dade76334f76d993b913d10bb2ac7,822486e25be33911a62b3f12e549d24d,ed37d73c29e3696f65817916316bf05d,197a95a09a55390eb435e35f7dab9f9f,59844ef9a5e469cd13b1208c4f1142d1,5befdbffa0268f49987f263afd421a6e,0.700000
2,e904a955c0350ca5b1b6bb84174ee5be,3b050c1502af19554bb9ef8efc67c00d,a17aab4730e49f8e2972bf26227b890c,868bbeb92bd33a728dcf77d0073cd44d,c4d98c14bd48ec0495239c1a65a5178c,4ea79c3619515a39423851b5ceca965e,d76c868bfb392d89b3ef222907de69fc,NaN,NaN
3,6650e87fecc3e969c0d48f7f5e9b2c81,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,e29a8f57507d7e4a926a6bd3d51841d3,f6c0c5501b42f208e094de2414ca1167,NaN,NaN
4,c10efc962e6b703053c2b44b7d48e8da,fd96fd2df80abfae763c3693932f8b26,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,f5606954ce38e9948ed7099251d7e339,995e60ea01e4f371dabd785e76aa5e2f,b3ac505ffec601de591ce2ce2e010d30,NaN,NaN
...,...,...,...,...,...,...,...,...,...
572254,0f59db4259ec62fcf00399b69786a3c0,cb6e848eff5f656c47a4541e6be72079,3c66c29e5c26efc705c261732b977950,a0c8441cbe902320c846da76329a38e6,5d043bbef0ca3cf5385432478f3d3751,3b48c6883b4d16ddc07cb64b7be38d1c,546da46c864f38bb193940a7b8704f44,3c66c29e5c26efc705c261732b977950,1.000000
572255,477ecc7fb10644c0d31408030cd464c6,aa7ce52650c7e4f7979e087cedb89020,b4b1c6c0e7395c50f2cf0702dd1f85d6,dd45bfa90e9186681877c15eb0166a35,c366d245501f007629813a28c9267d11,f4aabaca763955330cdf57f763d2a72c,f4aabaca763955330cdf57f763d2a72c,aa7ce52650c7e4f7979e087cedb89020,0.285714
572256,9e65aa27a1c3060b5909dda36859f39f,fba1bc1cf12692b1dd65c98e79985e52,261bceb1e3440067b85f5c54444d91be,65961a80125282df5c95b4c0a3606cc5,a91ec5b2b51524c53ce866f621208087,ab8683d279c095064693c282b8353b98,01c48d6a895a6efed0d5afba83b87902,NaN,NaN
572257,940ed872f8ae21d392495623c29df127,ceee2a16d7fb7531cdc633ac478fe46a,c31a95cdd5635c6edfbde9143286b9b7,aa41008b812435306a6fe831f40b0c11,d1492ba4a2fdf555510934e9740ea8c3,f1bc21ff88fe47bac896fbe591fa2f5f,d1492ba4a2fdf555510934e9740ea8c3,c31a95cdd5635c6edfbde9143286b9b7,0.750000


In [40]:
w = [1.0, 0.63, 0.5, 0.43, 0.38]
s = 0
for i in range(5):
    m = (x4[f'song_id_2{i+1}'] == x4['next2_song_id']).mean()
    print(i, m)
    s += m*w[i]
print(s*0.8)

0 0.21842382557548243
1 0.04763752077293673
2 0.046178391252911705
3 0.04120337120080243
4 0.041619266800522144
0.24404594423154552


In [42]:
# correct way to calculate score
import numpy as np
A = x4[[f'song_id_2{i+1}' for i in range(5)]].values == x4[['next2_song_id']].values
s = (A * np.array(w).reshape(1, -1)).max(axis=1).mean()
s

0.22794613977237577

In [43]:
a1 = test_source['session_id'].nunique()/meta_song.shape[0] * 0.2
a1

0.027760227881309232

In [44]:
print(s*0.8+a1*4 + x4['next2_song_id'].nunique()/meta_song.shape[0] * 0.2)

0.3054405666118683


In [46]:
x4['next2_song_id'].fillna(x4['song_id_1'], inplace=True)

In [47]:
w = np.array([1.0, 0.63, 0.5, 0.43, 0.38])
A = x4[[f'song_id_2{i+1}' for i in range(5)]].values == x4[['next2_song_id']].values
s = (A * w.reshape(1, -1)).max(axis=1).mean()
s

0.2494253301389756

In [48]:
print(s*0.8+a1*4 + x4['next2_song_id'].nunique()/meta_song.shape[0] * 0.2)

0.3395822933104136
